In [144]:
import pandas as pd
import json
import urllib
from urllib.request import urlopen
import re
import math
from datetime import datetime, timedelta

In [29]:
def refactor_values(data):
    new_data = {}
    for key, value in data.items():
            if value != '':
                new_data[key] = value
            if isinstance(value, str) and '<' in value:
                new_value = value.split('<')[1]
                new_data[key] = int(new_value)
    return new_data

In [248]:
url = "https://arsoxmlwrapper.app.grega.xyz/api/air/archive"

response = urlopen(url)
data_json = json.loads(response.read())

json_object = json.dumps(data_json)


In [249]:
save_file = open("data/raw/air", "w")  
json.dump(data_json, save_file)  
save_file.close()  

In [250]:
save_file = open("../data/raw/air", "w")  
json.dump(data_json, save_file)  
save_file.close()  

In [251]:
load_file = open('data/raw/air')
dataJson = json.load(load_file)
load_file.close()

In [ ]:
#----- meteo

In [252]:
url = "https://archive-api.open-meteo.com/v1/era5?latitude=46.568683&longitude=15.631008&start_date=2023-02-22&end_date=2023-03-16&hourly=temperature_2m,relativehumidity_2m,windspeed_10m"
response = urlopen(url)
data_json_weather = json.loads(response.read())

json_object_weather = json.dumps(data_json_weather)

In [253]:
save_file = open("data/raw/weather", "w")  
json.dump(data_json_weather, save_file)  
save_file.close()  

In [254]:
save_file = open("../data/raw/weather", "w")  
json.dump(data_json_weather, save_file)  
save_file.close()  

In [255]:
load_file = open('data/raw/weather')
dataJsonWeather = json.load(load_file)
load_file.close()

In [219]:
dataJsonWeather

{'latitude': 46.699997,
 'longitude': 15.699997,
 'generationtime_ms': 70.24705410003662,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 280.0,
 'hourly_units': {'time': 'iso8601',
  'temperature_2m': '°C',
  'relativehumidity_2m': '%',
  'windspeed_10m': 'km/h'},
 'hourly': {'time': ['2023-02-22T00:00',
   '2023-02-22T01:00',
   '2023-02-22T02:00',
   '2023-02-22T03:00',
   '2023-02-22T04:00',
   '2023-02-22T05:00',
   '2023-02-22T06:00',
   '2023-02-22T07:00',
   '2023-02-22T08:00',
   '2023-02-22T09:00',
   '2023-02-22T10:00',
   '2023-02-22T11:00',
   '2023-02-22T12:00',
   '2023-02-22T13:00',
   '2023-02-22T14:00',
   '2023-02-22T15:00',
   '2023-02-22T16:00',
   '2023-02-22T17:00',
   '2023-02-22T18:00',
   '2023-02-22T19:00',
   '2023-02-22T20:00',
   '2023-02-22T21:00',
   '2023-02-22T22:00',
   '2023-02-22T23:00',
   '2023-02-23T00:00',
   '2023-02-23T01:00',
   '2023-02-23T02:00',
   '2023-02-23T03:00',
   '2023-02-23T04:00',
   '

In [220]:
df_weather = pd.DataFrame.from_dict(json.loads(json.dumps(dataJsonWeather["hourly"])))

In [221]:
df_weather

,time,temperature_2m,relativehumidity_2m,windspeed_10m
0,2023-02-22T00:00,5.8,76.0,2.3
1,2023-02-22T01:00,5.1,78.0,1.8
2,2023-02-22T02:00,4.3,81.0,2.9
3,2023-02-22T03:00,3.6,83.0,4.0
4,2023-02-22T04:00,2.7,87.0,4.0
...,...,...,...,...
547,2023-03-16T19:00,NaN,NaN,NaN
548,2023-03-16T20:00,NaN,NaN,NaN
549,2023-03-16T21:00,NaN,NaN,NaN
550,2023-03-16T22:00,NaN,NaN,NaN


In [72]:
df_weather['time'] = pd.to_datetime(df_weather['time'], format='%Y-%m-%d %H')

In [73]:
df_weather

,time,temperature_2m,relativehumidity_2m,windspeed_10m
0,2023-02-22 00:00:00,5.8,76.0,2.3
1,2023-02-22 01:00:00,5.1,78.0,1.8
2,2023-02-22 02:00:00,4.3,81.0,2.9
3,2023-02-22 03:00:00,3.6,83.0,4.0
4,2023-02-22 04:00:00,2.7,87.0,4.0
...,...,...,...,...
547,2023-03-16 19:00:00,NaN,NaN,NaN
548,2023-03-16 20:00:00,NaN,NaN,NaN
549,2023-03-16 21:00:00,NaN,NaN,NaN
550,2023-03-16 22:00:00,NaN,NaN,NaN


In [256]:
df_weather.to_csv('../data/processed/processed_weather.csv', index=False)
df_weather.to_csv('data/processed/processed_weather.csv', index=False)
df_weather.to_csv('../src/data/data/processed/processed_weather.csv', index=False)

In [36]:
df = pd.DataFrame()

for i in range(len(dataJson)):
    data = dataJson[i]
    dictData = json.loads(data['json'])
    
    df1 = pd.json_normalize(dictData['arsopodatki']['postaja'])
    df = pd.concat([df, df1])#, axis=1, join="inner")

In [74]:
df = df.reset_index(drop=True)

In [75]:
df.dtypes

pm2.5                    object
nadm_visina               int64
datum_od         datetime64[ns]
merilno_mesto            object
o3                       object
benzen                   object
ge_sirina               float64
pm10                     object
co                       object
no2                      object
sifra                    object
datum_do                 object
ge_dolzina              float64
so2                      object
dtype: object

In [76]:
df.head()

,pm2.5,nadm_visina,datum_od,merilno_mesto,o3,benzen,ge_sirina,pm10,co,no2,sifra,datum_do,ge_dolzina,so2
0,59.0,299,2023-02-22 02:00:00,LJ Bežigrad,8.0,,46.065851,64.0,0.6,29,E403,2023-02-22 03:00,14.517454,NaN
1,59.0,305,2023-02-22 02:00:00,LJ Celovška,NaN,NaN,46.072399,62.0,NaN,30,E405,2023-02-22 03:00,14.491849,NaN
2,48.0,293,2023-02-22 02:00:00,LJ Vič,NaN,NaN,46.037791,51.0,NaN,NaN,E404,2023-02-22 03:00,14.494001,NaN
3,38.0,388,2023-02-22 02:00:00,Kranj,NaN,NaN,46.242115,41.0,NaN,NaN,E417,2023-02-22 03:00,14.366963,NaN
4,16.0,270,2023-02-22 02:00:00,MB Titova,NaN,,46.559202,19.0,NaN,17,E407,2023-02-22 03:00,15.656191,NaN


In [80]:
df_filtered = df.loc[df['merilno_mesto'] == 'MB Vrbanski'].copy()

In [81]:
df_filtered

,pm2.5,nadm_visina,datum_od,merilno_mesto,o3,benzen,ge_sirina,pm10,co,no2,sifra,datum_do,ge_dolzina,so2
5,17.0,279,2023-02-22 02:00:00,MB Vrbanski,33.0,NaN,46.568683,21.0,NaN,7,E408,2023-02-22 03:00,15.631008,NaN
26,21.0,279,2023-02-17 17:00:00,MB Vrbanski,,NaN,46.568683,29.0,NaN,18,E408,2023-02-17 18:00,15.631008,NaN
47,25.0,279,2023-02-23 10:00:00,MB Vrbanski,32,NaN,46.568683,30.0,NaN,10,E408,2023-02-23 11:00,15.631008,NaN
68,57.0,279,2023-02-15 23:00:00,MB Vrbanski,14,NaN,46.568683,60.0,NaN,10,E408,2023-02-16 00:00,15.631008,NaN
89,22.0,279,2023-02-17 20:00:00,MB Vrbanski,79,NaN,46.568683,25.0,NaN,11,E408,2023-02-17 21:00,15.631008,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174494,4,279,2023-03-16 09:00:00,MB Vrbanski,72,NaN,46.568683,5.0,NaN,4,E408,2023-03-16 10:00,15.631008,NaN
174515,4.0,279,2023-03-16 08:00:00,MB Vrbanski,68,NaN,46.568683,6.0,NaN,5,E408,2023-03-16 09:00,15.631008,NaN
174536,4,279,2023-03-16 10:00:00,MB Vrbanski,76,NaN,46.568683,5.0,NaN,,E408,2023-03-16 11:00,15.631008,NaN
174557,3,279,2023-03-16 17:00:00,MB Vrbanski,85,NaN,46.568683,5,NaN,3,E408,2023-03-16 18:00,15.631008,NaN


In [82]:
df_filtered['datum_od'] = pd.to_datetime(df_filtered['datum_od'], format='%Y-%m-%d %H')

In [83]:
df_filtered['datum_od']

5        2023-02-22 02:00:00
26       2023-02-17 17:00:00
47       2023-02-23 10:00:00
68       2023-02-15 23:00:00
89       2023-02-17 20:00:00
                 ...        
174494   2023-03-16 09:00:00
174515   2023-03-16 08:00:00
174536   2023-03-16 10:00:00
174557   2023-03-16 17:00:00
174578   2023-03-16 09:00:00
Name: datum_od, Length: 8314, dtype: datetime64[ns]

In [84]:
df_air = df_filtered[['datum_od', 'pm10']].copy()

In [85]:
df_air

,datum_od,pm10
5,2023-02-22 02:00:00,21.0
26,2023-02-17 17:00:00,29.0
47,2023-02-23 10:00:00,30.0
68,2023-02-15 23:00:00,60.0
89,2023-02-17 20:00:00,25.0
...,...,...
174494,2023-03-16 09:00:00,5.0
174515,2023-03-16 08:00:00,6.0
174536,2023-03-16 10:00:00,5.0
174557,2023-03-16 17:00:00,5


In [130]:
df_air.to_csv('../data/processed/data_air.csv', index=False)
df_air.to_csv('data/processed/data_air.csv', index=False)
df_air.to_csv('../src/data/data/processed/data_air.csv', index=False)

In [86]:
merged_df = pd.merge(df_air, df_weather, left_on='datum_od', right_on='time', how='inner')

In [87]:
merged_df

,datum_od,pm10,time,temperature_2m,relativehumidity_2m,windspeed_10m
0,2023-02-22 02:00:00,21.0,2023-02-22 02:00:00,4.3,81.0,2.9
1,2023-02-22 02:00:00,21.0,2023-02-22 02:00:00,4.3,81.0,2.9
2,2023-02-22 02:00:00,21.0,2023-02-22 02:00:00,4.3,81.0,2.9
3,2023-02-22 02:00:00,21.0,2023-02-22 02:00:00,4.3,81.0,2.9
4,2023-02-22 02:00:00,21.0,2023-02-22 02:00:00,4.3,81.0,2.9
...,...,...,...,...,...,...
6543,2023-03-16 16:00:00,5,2023-03-16 16:00:00,NaN,NaN,NaN
6544,2023-03-16 16:00:00,5,2023-03-16 16:00:00,NaN,NaN,NaN
6545,2023-03-16 16:00:00,5,2023-03-16 16:00:00,NaN,NaN,NaN
6546,2023-03-16 16:00:00,5,2023-03-16 16:00:00,NaN,NaN,NaN


In [102]:
merged_unique_df = merged_df.drop_duplicates()

In [103]:
merged_unique_df

,datum_od,pm10,time,temperature_2m,relativehumidity_2m,windspeed_10m
0,2023-02-22 02:00:00,21.0,2023-02-22 02:00:00,4.3,81.0,2.9
12,2023-02-23 10:00:00,30.0,2023-02-23 10:00:00,8.4,76.0,2.2
24,2023-03-11 11:00:00,5,2023-03-11 11:00:00,NaN,NaN,NaN
36,2023-03-07 00:00:00,42,2023-03-07 00:00:00,2.7,95.0,8.7
48,2023-03-13 18:00:00,22,2023-03-13 18:00:00,NaN,NaN,NaN
...,...,...,...,...,...,...
6488,2023-03-16 12:00:00,4.0,2023-03-16 12:00:00,NaN,NaN,NaN
6500,2023-03-16 08:00:00,6.0,2023-03-16 08:00:00,NaN,NaN,NaN
6512,2023-03-16 09:00:00,5.0,2023-03-16 09:00:00,NaN,NaN,NaN
6524,2023-03-16 10:00:00,5.0,2023-03-16 10:00:00,NaN,NaN,NaN


In [104]:
merged_unique_df = merged_unique_df.rename(columns={'datum_od': 'date'})

In [105]:
merged_unique_df

,date,pm10,time,temperature_2m,relativehumidity_2m,windspeed_10m
0,2023-02-22 02:00:00,21.0,2023-02-22 02:00:00,4.3,81.0,2.9
12,2023-02-23 10:00:00,30.0,2023-02-23 10:00:00,8.4,76.0,2.2
24,2023-03-11 11:00:00,5,2023-03-11 11:00:00,NaN,NaN,NaN
36,2023-03-07 00:00:00,42,2023-03-07 00:00:00,2.7,95.0,8.7
48,2023-03-13 18:00:00,22,2023-03-13 18:00:00,NaN,NaN,NaN
...,...,...,...,...,...,...
6488,2023-03-16 12:00:00,4.0,2023-03-16 12:00:00,NaN,NaN,NaN
6500,2023-03-16 08:00:00,6.0,2023-03-16 08:00:00,NaN,NaN,NaN
6512,2023-03-16 09:00:00,5.0,2023-03-16 09:00:00,NaN,NaN,NaN
6524,2023-03-16 10:00:00,5.0,2023-03-16 10:00:00,NaN,NaN,NaN


In [106]:
koncni_df = merged_unique_df.drop(['time'], axis=1)

In [107]:
koncni_df

,date,pm10,temperature_2m,relativehumidity_2m,windspeed_10m
0,2023-02-22 02:00:00,21.0,4.3,81.0,2.9
12,2023-02-23 10:00:00,30.0,8.4,76.0,2.2
24,2023-03-11 11:00:00,5,NaN,NaN,NaN
36,2023-03-07 00:00:00,42,2.7,95.0,8.7
48,2023-03-13 18:00:00,22,NaN,NaN,NaN
...,...,...,...,...,...
6488,2023-03-16 12:00:00,4.0,NaN,NaN,NaN
6500,2023-03-16 08:00:00,6.0,NaN,NaN,NaN
6512,2023-03-16 09:00:00,5.0,NaN,NaN,NaN
6524,2023-03-16 10:00:00,5.0,NaN,NaN,NaN


In [246]:
####

In [247]:
#file = open('../data/raw/data.json', 'r', encoding='utf-8')
#raw = json.load(file)
#file.close()



#df = pd.DataFrame()

#for i in range(len(raw)):
#    jdata = json.loads(raw[i]['json'])
#    station = jdata['arsopodatki']['postaja']
#    for i in range(len(station)):
#        data = station[i]
#        data = refactor_values(data)
#        df = pd.concat([df, pd.json_normalize(data)])  
        
#df

In [248]:
####

In [108]:
koncni_df.dtypes

date                   datetime64[ns]
pm10                           object
temperature_2m                float64
relativehumidity_2m           float64
windspeed_10m                 float64
dtype: object

In [109]:
print(koncni_df.isnull().sum())

date                     0
pm10                     0
temperature_2m         162
relativehumidity_2m    162
windspeed_10m          162
dtype: int64


In [114]:
koncni_df['temperature_2m'].fillna(koncni_df['temperature_2m'].mean(), inplace=True)
koncni_df['relativehumidity_2m'].fillna(koncni_df['relativehumidity_2m'].mean(), inplace=True)
koncni_df['windspeed_10m'].fillna(koncni_df['windspeed_10m'].mean(), inplace=True)

In [115]:
print(koncni_df.isnull().sum())

date                   0
pm10                   0
temperature_2m         0
relativehumidity_2m    0
windspeed_10m          0
dtype: int64


In [111]:
#cat_cols = koncni_df.select_dtypes(include=['object']).columns
#print(cat_cols)
#num_cols = koncni_df.select_dtypes(include=['number']).columns
#print(num_cols)

Index(['pm10'], dtype='object')
Index(['temperature_2m', 'relativehumidity_2m', 'windspeed_10m'], dtype='object')


In [306]:
#for col in num_cols:
#    df[col].fillna((df[col].mean()), inplace=True)

#df_location = pd.get_dummies(df['merilno_mesto'])
#df = pd.concat([df, df_location], axis=1).reindex(df.index)
#df.pop('merilno_mesto')
#df.pop('sifra')
#df.pop('datum_od')
#df.pop('datum_do')

#df

,pm2.5,nadm_visina,o3,benzen,ge_sirina,pm10,co,no2,ge_dolzina,so2,...,MB Vrbanski,MS Cankarjeva,MS Rakičan,NG Grčna,Novo mesto,Otlica,Ptuj,Rečica v I.Bistrici,Trbovlje,Zagorje
0,34.0,299,61,,46.065851,39.0,0.3,19,14.517454,NaN,...,0,0,0,0,0,0,0,0,0,0
1,38.0,305,NaN,NaN,46.072399,46.0,NaN,38,14.491849,NaN,...,0,0,0,0,0,0,0,0,0,0
2,35.0,293,NaN,NaN,46.037791,40.0,NaN,NaN,14.494001,NaN,...,0,0,0,0,0,0,0,0,0,0
3,88.0,388,NaN,NaN,46.242115,95.0,NaN,NaN,14.366963,NaN,...,0,0,0,0,0,0,0,0,0,0
4,33.0,270,NaN,2.3,46.559202,41.0,NaN,32,15.656191,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152035,26,241,17,NaN,46.131416,30,NaN,30,15.000906,4.0,...,0,0,0,0,0,0,0,0,0,1
152036,14,290,NaN,NaN,46.144213,15,NaN,NaN,15.088097,NaN,...,0,0,0,0,0,0,0,0,0,0
152037,41,220,9,NaN,45.802080,44,NaN,19,15.182060,NaN,...,0,0,0,0,1,0,0,0,0,0
152038,10,540,36,,45.561344,13,NaN,4,14.862816,2.0,...,0,0,0,0,0,0,0,0,0,0


In [116]:
koncni_df.dtypes

date                   datetime64[ns]
pm10                           object
temperature_2m                float64
relativehumidity_2m           float64
windspeed_10m                 float64
dtype: object

In [117]:
stevilski = koncni_df[(["pm10"])]

In [118]:
stevilski = stevilski.apply(pd.to_numeric, args=('coerce',))

In [119]:
stevilski.isnull().sum()

pm10    6
dtype: int64

In [120]:
stevilski['pm10'].fillna(stevilski['pm10'].mean(), inplace=True)
stevilski.isnull().sum()

pm10    0
dtype: int64

In [121]:
df_drop = koncni_df.drop(columns=stevilski)

In [122]:
koncni_df = pd.concat([stevilski, df_drop], axis=1, join="inner")

In [124]:
koncni_df.dtypes

pm10                          float64
date                   datetime64[ns]
temperature_2m                float64
relativehumidity_2m           float64
windspeed_10m                 float64
dtype: object

In [131]:
koncni_df.to_csv('../data/processed/data_weather.csv', index=False)

In [132]:
koncni_df.to_csv('data/processed/data_weather.csv', index=False)

In [133]:
koncni_df.to_csv('../src/data/data/processed/data_weather.csv', index=False)

In [ ]:
#Probam za forecast

In [222]:
url = "https://api.open-meteo.com/v1/forecast?latitude=46.5686&longitude=15.631&hourly=temperature_2m,relativehumidity_2m,windspeed_10m"
response = urlopen(url)
data_json_forecast = json.loads(response.read())

json_object_forecast = json.dumps(data_json_forecast)

In [223]:
save_file = open("data/raw/forecast", "w")  
json.dump(data_json_forecast, save_file)  
save_file.close()  

In [224]:
load_file = open('data/raw/forecast')
dataJsonWeatherForecast = json.load(load_file)
load_file.close()


In [238]:
dataJsonWeatherForecast["hourly"]

{'time': ['2023-03-17T00:00',
  '2023-03-17T01:00',
  '2023-03-17T02:00',
  '2023-03-17T03:00',
  '2023-03-17T04:00',
  '2023-03-17T05:00',
  '2023-03-17T06:00',
  '2023-03-17T07:00',
  '2023-03-17T08:00',
  '2023-03-17T09:00',
  '2023-03-17T10:00',
  '2023-03-17T11:00',
  '2023-03-17T12:00',
  '2023-03-17T13:00',
  '2023-03-17T14:00',
  '2023-03-17T15:00',
  '2023-03-17T16:00',
  '2023-03-17T17:00',
  '2023-03-17T18:00',
  '2023-03-17T19:00',
  '2023-03-17T20:00',
  '2023-03-17T21:00',
  '2023-03-17T22:00',
  '2023-03-17T23:00',
  '2023-03-18T00:00',
  '2023-03-18T01:00',
  '2023-03-18T02:00',
  '2023-03-18T03:00',
  '2023-03-18T04:00',
  '2023-03-18T05:00',
  '2023-03-18T06:00',
  '2023-03-18T07:00',
  '2023-03-18T08:00',
  '2023-03-18T09:00',
  '2023-03-18T10:00',
  '2023-03-18T11:00',
  '2023-03-18T12:00',
  '2023-03-18T13:00',
  '2023-03-18T14:00',
  '2023-03-18T15:00',
  '2023-03-18T16:00',
  '2023-03-18T17:00',
  '2023-03-18T18:00',
  '2023-03-18T19:00',
  '2023-03-18T20:00',
  

In [225]:
df_weather_forecast = pd.DataFrame.from_dict(json.loads(json.dumps(dataJsonWeatherForecast["hourly"])))

In [227]:
df_weather_forecast

,time,temperature_2m,relativehumidity_2m,windspeed_10m
0,2023-03-17T00:00,-0.1,72,1.3
1,2023-03-17T01:00,-1.2,76,2.1
2,2023-03-17T02:00,-1.9,77,1.6
3,2023-03-17T03:00,-2.4,77,2.6
4,2023-03-17T04:00,-1.4,71,0.5
...,...,...,...,...
163,2023-03-23T19:00,12.9,59,6.5
164,2023-03-23T20:00,11.9,63,5.7
165,2023-03-23T21:00,10.9,67,4.9
166,2023-03-23T22:00,10.4,68,4.9


In [229]:
df_weather_forecast['time'] = pd.to_datetime(df_weather_forecast['time'], format='%Y-%m-%d %H')

In [237]:
df_weather_forecast

,time,temperature_2m,relativehumidity_2m,windspeed_10m
0,2023-03-17 00:00:00,-0.1,72,1.3
1,2023-03-17 01:00:00,-1.2,76,2.1
2,2023-03-17 02:00:00,-1.9,77,1.6
3,2023-03-17 03:00:00,-2.4,77,2.6
4,2023-03-17 04:00:00,-1.4,71,0.5
...,...,...,...,...
163,2023-03-23 19:00:00,12.9,59,6.5
164,2023-03-23 20:00:00,11.9,63,5.7
165,2023-03-23 21:00:00,10.9,67,4.9
166,2023-03-23 22:00:00,10.4,68,4.9


In [239]:
df_weather_forecast.dtypes

time                   datetime64[ns]
temperature_2m                float64
relativehumidity_2m             int64
windspeed_10m                 float64
dtype: object

In [243]:
now = datetime.now()
now_str = now.strftime("%Y-%m-%d %H")
now = datetime.strptime(now_str, "%Y-%m-%d %H")
hour = now + timedelta(hours=1)
hour_str = hour.strftime("%Y-%m-%d %H")
print(hour_str)

2023-03-17 20


In [245]:
df_forecast_filtered = df_weather_forecast.loc[df_weather_forecast['time'] == hour_str].copy()

In [258]:
df_forecast = df_forecast_filtered.drop('time', axis=1)

In [259]:
df_forecast

,temperature_2m,relativehumidity_2m,windspeed_10m
20,6.4,47,6.9


In [266]:
data_dict = df_forecast.to_dict(orient='records')[0]
print(data_dict)

{'temperature_2m': 6.4, 'relativehumidity_2m': 47, 'windspeed_10m': 6.9}


In [262]:
df_forecast.to_csv('../data/processed/data_forecast.csv', index=False)
df_forecast.to_csv('data/processed/data_forecast.csv', index=False)
df_forecast.to_csv('../src/data/data/processed/data_forecast.csv', index=False)

In [ ]:
#Tu dalje ne rabim več, ker ni blo prav in nimam več prometa

In [364]:
df.columns

Index(['pm2.5', 'nadm_visina', 'o3', 'benzen', 'ge_sirina', 'pm10', 'co',
       'no2', 'ge_dolzina', 'so2', 'CE Ljubljanska', 'CE bolnica', 'Hrastnik',
       'Iskrba', 'Koper', 'Kranj', 'Krvavec', 'LJ Bežigrad', 'LJ Celovška',
       'LJ Vič', 'MB Titova', 'MB Vrbanski', 'MS Cankarjeva', 'MS Rakičan',
       'NG Grčna', 'Novo mesto', 'Otlica', 'Ptuj', 'Rečica v I.Bistrici',
       'Trbovlje', 'Zagorje', 'promet'],
      dtype='object')

In [365]:
df.dtypes

pm2.5                  float64
nadm_visina              int64
o3                     float64
benzen                 float64
ge_sirina              float64
pm10                   float64
co                     float64
no2                    float64
ge_dolzina             float64
so2                    float64
CE Ljubljanska           uint8
CE bolnica               uint8
Hrastnik                 uint8
Iskrba                   uint8
Koper                    uint8
Kranj                    uint8
Krvavec                  uint8
LJ Bežigrad              uint8
LJ Celovška              uint8
LJ Vič                   uint8
MB Titova                uint8
MB Vrbanski              uint8
MS Cankarjeva            uint8
MS Rakičan               uint8
NG Grčna                 uint8
Novo mesto               uint8
Otlica                   uint8
Ptuj                     uint8
Rečica v I.Bistrici      uint8
Trbovlje                 uint8
Zagorje                  uint8
promet                 float64
dtype: o

In [366]:
####

In [367]:
#PROMET - naloga 2

In [368]:
df_promet = pd.read_csv("data/raw/promet.csv", sep = ";", decimal = ".", encoding = "cp1250")
df_promet.head()

,Kat. ceste,Štev. ceste,Štev. odseka,Prometni odsek,Stac. začetka,Stac. konca,Števno mesto,Ime števnega mesta,Tip štetja,Vsa vozila (PLDP),Motorji,Osebna vozila,Avtobusi,"Lah. tov. < 3,5t","Sr. tov. 3,5-7t",Tež. tov. nad 7t,Tov. s prik.,Vlačilci,Dnevni NOO,Tip
0,AC,A1,30.0,MP ŠENTILJ - POČIVALIŠČE ŠENTILJ,0.0,482.000,674.0,MT Šentilj AC,QLTC8,18.686,34,12.884,47.0,2.030,226.0,126.0,407.0,2.932,2.816,PLDP
1,AC,A1,30.0,POČIVALIŠČE ŠENTILJ - ŠENTILJ,482.0,1.137,869.0,Šentilj AC,QLTC8,18.560,34,12.789,47.0,2.013,225.0,126.0,405.0,2.921,2.805,PLDP
2,AC,A1,199.0,PRIKLJ.ŠENTILJ S,0.0,842.000,868.0,Priklj. Šentilj S,QLTC8,284.000,1,239.000,0.0,26.000,2.0,1.0,2.0,13.000,13.000,PLDP
3,AC,A1,31.0,ŠENTILJ - PESNICA,0.0,9.761,837.0,Pesnica AC,QLTC8,25.324,49,18.784,80.0,2.707,204.0,94.0,485.0,2.921,2.875,PLDP
4,AC,A1,65.0,PESNICA - DRAGUČOVA,0.0,2.570,NaN,NaN,P,25.500,60,18.735,85.0,2.800,210.0,110.0,600.0,2.900,2.961,PLDP


In [369]:
#df_promet_filtered = df_promet[df_promet["Prometni odsek"].str.contains("CELJE|HRASTNIK|ISKRBA|KOPER|KRANJ|KRVAVEC|LJUBLJANA|BEŽIGRAD|CELOVŠKA|VIČ|MARIBOR|TITOVA|VRBANSKI|MURSKA SOBOTA|CANKARJEVA|RAKIČANNOVA GORICA|GRČNA|NOVO MESTO|OTILICA|PTUJ|REČICA|IL.|TRBOVLJE|ZAGORJE")==True]
df_promet_filtered = df_promet[df_promet["Prometni odsek"].str.contains("CELJE|HRASTNIK|ISKRBA|KOPER|KRANJ|KRVAVEC|LJUBLJANA|BEŽIGRAD|CELOVŠKA|VIČ|MARIBOR|TITOVA|VRBANSKI|MURSKA SOBOTA|CANKARJEVA|RAKIČAN|NOVA GORICA|GRČNA|NOVO MESTO|OTLICA|PTUJ|REČICA|IL\.|TRBOVLJE|ZAGORJE")==True]

df_promet_filtered

,Kat. ceste,Štev. ceste,Štev. odseka,Prometni odsek,Stac. začetka,Stac. konca,Števno mesto,Ime števnega mesta,Tip štetja,Vsa vozila (PLDP),Motorji,Osebna vozila,Avtobusi,"Lah. tov. < 3,5t","Sr. tov. 3,5-7t",Tež. tov. nad 7t,Tov. s prik.,Vlačilci,Dnevni NOO,Tip
6,AC,A1,71.0,MB (ZRKOVSKA) - MB (PTUJSKA),4.452,7.744,NaN,NaN,P,35.000,60,22.530,190.0,4.200,620.0,250.0,1.1,6.050,6.127,PLDP
7,AC,A1,66.0,MB (PTUJSKA) - ROGOZA,0.000,2.590,840.0,Ptujska AC,QLTC8,41.976,87,28.404,157.0,5.139,626.0,299.0,912.0,6.352,6.214,PLDP
24,AC,A1,39.0,LJUBEČNA - CELJE,6.436,8.438,1030.0,Celje Center-V AC,QLTC8,38.286,87,25.912,131.0,4.906,665.0,381.0,801.0,5.403,5.388,PLDP
25,AC,A1,40.0,CELJE - CELJE ZAHOD,0.000,5.261,648.0,Lopata AC,QLTC8,42.302,97,29.020,139.0,5.475,784.0,418.0,874.0,5.495,5.574,PLDP
26,AC,A1,40.0,CELJE ZAHOD - ARJA VAS,5.261,7.509,1031.0,Arja vas AC,QLTC8,47.450,120,33.578,156.0,5.694,886.0,483.0,892.0,5.641,5.770,PLDP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,R3,713,4910.0,KRIŽ.SPUHLJA - PTUJ,28.500,31.230,425.0,Rogoznica,QLTC10,9.928,57,8.990,22.0,653.000,87.0,70.0,21.0,28.000,168.000,WIM
1102,R3,741,1487.0,DEKANI - PRIKLJ. LUKA KOPER,0.000,2.215,NaN,NaN,P,2.200,20,2.048,60.0,25.000,25.0,15.0,2.0,5.000,42.000,PLDP
1121,R3,751,5507.0,NIZKA - REČICA OB SAVINJI,0.000,905.000,NaN,NaN,P,1.700,10,1.310,50.0,150.000,60.0,50.0,30.0,40.000,102.000,PLDP
1152,RT,915,363.0,IL.BISTRICA (DO PODGRAJSKE UL.),0.000,182.000,NaN,NaN,P,8.500,110,7.785,75.0,400.000,65.0,20.0,20.0,25.000,83.000,PLDP


In [370]:
#df_promet_filtered = df_promet_filtered.drop(['Kat. ceste', 'Štev. ceste', 'Štev. odseka', 'Stac. začetka', 'Stac. konca',
#                                              'Števno mesto', 'Ime števnega mesta', 'Tip   štetja',
#                                             '', '', '', ''], axis=1)

In [371]:
df_promet_filtered = df_promet_filtered[['Prometni odsek', 'Vsa vozila (PLDP)']]

In [372]:
df_promet_filtered

,Prometni odsek,Vsa vozila (PLDP)
6,MB (ZRKOVSKA) - MB (PTUJSKA),35.000
7,MB (PTUJSKA) - ROGOZA,41.976
24,LJUBEČNA - CELJE,38.286
25,CELJE - CELJE ZAHOD,42.302
26,CELJE ZAHOD - ARJA VAS,47.450
...,...,...
1041,KRIŽ.SPUHLJA - PTUJ,9.928
1102,DEKANI - PRIKLJ. LUKA KOPER,2.200
1121,NIZKA - REČICA OB SAVINJI,1.700
1152,IL.BISTRICA (DO PODGRAJSKE UL.),8.500


In [373]:
df_promet_filtered.dtypes

Prometni odsek        object
Vsa vozila (PLDP)    float64
dtype: object

In [374]:
cat_cols = df_promet_filtered.select_dtypes(include=['object']).columns
print(cat_cols)
num_cols = df_promet_filtered.select_dtypes(include=['number']).columns
print(num_cols)

Index(['Prometni odsek'], dtype='object')
Index(['Vsa vozila (PLDP)'], dtype='object')


In [375]:
df_promet_celje = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("CELJE")==True]
df_promet_celje = df_promet_celje["Vsa vozila (PLDP)"].sum()
df_promet_celje

221.894

In [376]:
df_promet_hrastnik = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("HRASTNIK")==True]
df_promet_hrastnik = df_promet_hrastnik["Vsa vozila (PLDP)"].sum()
df_promet_hrastnik

285.955

In [377]:
df_promet_iskrba = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("ISKRBA")==True]
df_promet_iskrba = df_promet_iskrba["Vsa vozila (PLDP)"].sum()
df_promet_iskrba

0.0

In [378]:
df_promet_koper = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("KOPER")==True]
df_promet_koper = df_promet_koper["Vsa vozila (PLDP)"].sum()
df_promet_koper

113.777

In [379]:
df_promet_kranj = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("KRANJ")==True]
df_promet_kranj = df_promet_kranj["Vsa vozila (PLDP)"].sum()
df_promet_kranj

215.248

In [380]:
df_promet_krvavec = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("KRVAVEC")==True]
df_promet_krvavec = df_promet_krvavec["Vsa vozila (PLDP)"].sum()
df_promet_krvavec

0.0

In [381]:
df_promet_jubljana = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("LJUBLJANA|BEŽIGRAD|CELOVŠKA|VIČ")==True]
df_promet_jubljana = df_promet_jubljana["Vsa vozila (PLDP)"].sum()
df_promet_jubljana

1326.4769999999999

In [382]:
df_promet_maribor = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("MARIBOR|TITOVA|VRBANSKI")==True]
df_promet_maribor = df_promet_maribor["Vsa vozila (PLDP)"].sum()
df_promet_maribor

70.612

In [383]:
df_promet_murska_sobota = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("MURSKA SOBOTA|CANKARJEVA|RAKIČAN")==True]
df_promet_murska_sobota = df_promet_murska_sobota["Vsa vozila (PLDP)"].sum()
df_promet_murska_sobota

74.1

In [384]:
df_promet_nova_gorica = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("NOVA GORICA|GRČNA")==True]
df_promet_nova_gorica = df_promet_nova_gorica["Vsa vozila (PLDP)"].sum()
df_promet_nova_gorica

37.199

In [385]:
df_promet_novo_mesto = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("NOVO MESTO")==True]
df_promet_novo_mesto = df_promet_novo_mesto["Vsa vozila (PLDP)"].sum()
df_promet_novo_mesto

90.955

In [386]:
df_promet_otlica = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("OTLICA")==True]
df_promet_otlica = df_promet_otlica["Vsa vozila (PLDP)"].sum()
df_promet_otlica

0.0

In [387]:
df_promet_ptuj = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("PTUJ")==True]
df_promet_ptuj = df_promet_ptuj["Vsa vozila (PLDP)"].sum()
df_promet_ptuj

212.28699999999998

In [388]:
df_promet_il_bistrica = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("REČICA|IL\.")==True]
df_promet_il_bistrica = df_promet_il_bistrica["Vsa vozila (PLDP)"].sum()
df_promet_il_bistrica

1247.455

In [389]:
df_promet_trbovlje = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("TRBOVLJE")==True]
df_promet_trbovlje = df_promet_trbovlje["Vsa vozila (PLDP)"].sum()
df_promet_trbovlje

298.169

In [390]:
df_promet_zagorje = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("ZAGORJE")==True]
df_promet_zagorje = df_promet_zagorje["Vsa vozila (PLDP)"].sum()
df_promet_zagorje

32.744

In [391]:
df2 = df

In [395]:
columns = ['pm2.5', 'nadm_visina', 'o3', 'benzen', 'ge_sirina', 'pm10', 'co',
       'no2', 'ge_dolzina', 'so2', 'CE Ljubljanska', 'CE bolnica', 'Hrastnik',
       'Iskrba', 'Koper', 'Kranj', 'Krvavec', 'LJ Bežigrad', 'LJ Celovška',
       'LJ Vič', 'MB Titova', 'MB Vrbanski', 'MS Cankarjeva', 'MS Rakičan',
       'NG Grčna', 'Novo mesto', 'Otlica', 'Ptuj', 'Rečica v I.Bistrici',
       'Trbovlje', 'Zagorje']
df2 = df.reindex(columns=columns)

In [396]:
df.dtypes

pm2.5                  float64
nadm_visina              int64
o3                     float64
benzen                 float64
ge_sirina              float64
pm10                   float64
co                     float64
no2                    float64
ge_dolzina             float64
so2                    float64
CE Ljubljanska           uint8
CE bolnica               uint8
Hrastnik                 uint8
Iskrba                   uint8
Koper                    uint8
Kranj                    uint8
Krvavec                  uint8
LJ Bežigrad              uint8
LJ Celovška              uint8
LJ Vič                   uint8
MB Titova                uint8
MB Vrbanski              uint8
MS Cankarjeva            uint8
MS Rakičan               uint8
NG Grčna                 uint8
Novo mesto               uint8
Otlica                   uint8
Ptuj                     uint8
Rečica v I.Bistrici      uint8
Trbovlje                 uint8
Zagorje                  uint8
promet                 float64
dtype: o

In [397]:
columns

['pm2.5',
 'nadm_visina',
 'o3',
 'benzen',
 'ge_sirina',
 'pm10',
 'co',
 'no2',
 'ge_dolzina',
 'so2',
 'CE Ljubljanska',
 'CE bolnica',
 'Hrastnik',
 'Iskrba',
 'Koper',
 'Kranj',
 'Krvavec',
 'LJ Bežigrad',
 'LJ Celovška',
 'LJ Vič',
 'MB Titova',
 'MB Vrbanski',
 'MS Cankarjeva',
 'MS Rakičan',
 'NG Grčna',
 'Novo mesto',
 'Otlica',
 'Ptuj',
 'Rečica v I.Bistrici',
 'Trbovlje',
 'Zagorje']

In [398]:
df2["promet"] = df2.apply(lambda row: df_promet_celje if row["CE Ljubljanska"] == 1
                          else df_promet_celje if row["CE bolnica"] == 1
                          else df_promet_hrastnik if row["Hrastnik"] == 1
                          else df_promet_iskrba if row["Iskrba"] == 1
                          else df_promet_koper if row["Koper"] == 1
                          else df_promet_kranj if row["Kranj"] == 1
                          else df_promet_krvavec if row["Krvavec"] == 1
                          else df_promet_jubljana if row["LJ Bežigrad"] == 1
                          else df_promet_jubljana if row["LJ Celovška"] == 1
                          else df_promet_jubljana if row["LJ Vič"] == 1
                          else df_promet_maribor if row["MB Titova"] == 1
                          else df_promet_maribor if row["MB Vrbanski"] == 1
                          else df_promet_murska_sobota if row["MS Cankarjeva"] == 1
                          else df_promet_murska_sobota if row["MS Rakičan"] == 1
                          else df_promet_nova_gorica if row["NG Grčna"] == 1
                          else df_promet_novo_mesto if row["Novo mesto"] == 1
                          else df_promet_otlica if row["Otlica"] == 1
                          else df_promet_ptuj if row["Ptuj"] == 1
                          else df_promet_il_bistrica if row["Rečica v I.Bistrici"] == 1
                          else df_promet_trbovlje if row["Trbovlje"] == 1
                          else df_promet_zagorje if row["Zagorje"] == 1
                          else None, axis=1)


In [399]:
df2

,pm2.5,nadm_visina,o3,benzen,ge_sirina,pm10,co,no2,ge_dolzina,so2,...,MS Cankarjeva,MS Rakičan,NG Grčna,Novo mesto,Otlica,Ptuj,Rečica v I.Bistrici,Trbovlje,Zagorje,promet
0,34.000000,299,61.000000,1.015454,46.065851,39.000000,0.300000,19.000000,14.517454,2.424805,...,0,0,0,0,0,0,0,0,0,1326.477
1,38.000000,305,54.802235,1.015454,46.072399,46.000000,0.401293,38.000000,14.491849,2.424805,...,0,0,0,0,0,0,0,0,0,1326.477
2,35.000000,293,54.802235,1.015454,46.037791,40.000000,0.401293,21.014164,14.494001,2.424805,...,0,0,0,0,0,0,0,0,0,1326.477
3,88.000000,388,54.802235,1.015454,46.242115,95.000000,0.401293,21.014164,14.366963,2.424805,...,0,0,0,0,0,0,0,0,0,215.248
4,33.000000,270,54.802235,2.300000,46.559202,41.000000,0.401293,32.000000,15.656191,2.424805,...,0,0,0,0,0,0,0,0,0,70.612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152035,26.000000,241,17.000000,1.015454,46.131416,30.000000,0.401293,30.000000,15.000906,4.000000,...,0,0,0,0,0,0,0,0,1,32.744
152036,14.000000,290,54.802235,1.015454,46.144213,15.000000,0.401293,21.014164,15.088097,2.424805,...,0,0,0,0,0,0,0,0,0,285.955
152037,41.000000,220,9.000000,1.015454,45.802080,44.000000,0.401293,19.000000,15.182060,2.424805,...,0,0,0,1,0,0,0,0,0,90.955
152038,10.000000,540,36.000000,1.015454,45.561344,13.000000,0.401293,4.000000,14.862816,2.000000,...,0,0,0,0,0,0,0,0,0,0.000


In [400]:
cat_cols_df2 = df2.select_dtypes(include=['object']).columns
print(cat_cols_df2)
num_cols_df2 = df2.select_dtypes(include=['number']).columns
print(num_cols_df2)

Index([], dtype='object')
Index(['pm2.5', 'nadm_visina', 'o3', 'benzen', 'ge_sirina', 'pm10', 'co',
       'no2', 'ge_dolzina', 'so2', 'CE Ljubljanska', 'CE bolnica', 'Hrastnik',
       'Iskrba', 'Koper', 'Kranj', 'Krvavec', 'LJ Bežigrad', 'LJ Celovška',
       'LJ Vič', 'MB Titova', 'MB Vrbanski', 'MS Cankarjeva', 'MS Rakičan',
       'NG Grčna', 'Novo mesto', 'Otlica', 'Ptuj', 'Rečica v I.Bistrici',
       'Trbovlje', 'Zagorje', 'promet'],
      dtype='object')


In [401]:
df2.dtypes

pm2.5                  float64
nadm_visina              int64
o3                     float64
benzen                 float64
ge_sirina              float64
pm10                   float64
co                     float64
no2                    float64
ge_dolzina             float64
so2                    float64
CE Ljubljanska           uint8
CE bolnica               uint8
Hrastnik                 uint8
Iskrba                   uint8
Koper                    uint8
Kranj                    uint8
Krvavec                  uint8
LJ Bežigrad              uint8
LJ Celovška              uint8
LJ Vič                   uint8
MB Titova                uint8
MB Vrbanski              uint8
MS Cankarjeva            uint8
MS Rakičan               uint8
NG Grčna                 uint8
Novo mesto               uint8
Otlica                   uint8
Ptuj                     uint8
Rečica v I.Bistrici      uint8
Trbovlje                 uint8
Zagorje                  uint8
promet                 float64
dtype: o

In [402]:
df2.isnull().sum()

pm2.5                  0
nadm_visina            0
o3                     0
benzen                 0
ge_sirina              0
pm10                   0
co                     0
no2                    0
ge_dolzina             0
so2                    0
CE Ljubljanska         0
CE bolnica             0
Hrastnik               0
Iskrba                 0
Koper                  0
Kranj                  0
Krvavec                0
LJ Bežigrad            0
LJ Celovška            0
LJ Vič                 0
MB Titova              0
MB Vrbanski            0
MS Cankarjeva          0
MS Rakičan             0
NG Grčna               0
Novo mesto             0
Otlica                 0
Ptuj                   0
Rečica v I.Bistrici    0
Trbovlje               0
Zagorje                0
promet                 0
dtype: int64

In [407]:
df2

,pm2.5,nadm_visina,o3,benzen,ge_sirina,pm10,co,no2,ge_dolzina,so2,...,MS Cankarjeva,MS Rakičan,NG Grčna,Novo mesto,Otlica,Ptuj,Rečica v I.Bistrici,Trbovlje,Zagorje,promet
0,34.000000,299,61.000000,1.015454,46.065851,39.000000,0.300000,19.000000,14.517454,2.424805,...,0,0,0,0,0,0,0,0,0,1326.477
1,38.000000,305,54.802235,1.015454,46.072399,46.000000,0.401293,38.000000,14.491849,2.424805,...,0,0,0,0,0,0,0,0,0,1326.477
2,35.000000,293,54.802235,1.015454,46.037791,40.000000,0.401293,21.014164,14.494001,2.424805,...,0,0,0,0,0,0,0,0,0,1326.477
3,88.000000,388,54.802235,1.015454,46.242115,95.000000,0.401293,21.014164,14.366963,2.424805,...,0,0,0,0,0,0,0,0,0,215.248
4,33.000000,270,54.802235,2.300000,46.559202,41.000000,0.401293,32.000000,15.656191,2.424805,...,0,0,0,0,0,0,0,0,0,70.612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152035,26.000000,241,17.000000,1.015454,46.131416,30.000000,0.401293,30.000000,15.000906,4.000000,...,0,0,0,0,0,0,0,0,1,32.744
152036,14.000000,290,54.802235,1.015454,46.144213,15.000000,0.401293,21.014164,15.088097,2.424805,...,0,0,0,0,0,0,0,0,0,285.955
152037,41.000000,220,9.000000,1.015454,45.802080,44.000000,0.401293,19.000000,15.182060,2.424805,...,0,0,0,1,0,0,0,0,0,90.955
152038,10.000000,540,36.000000,1.015454,45.561344,13.000000,0.401293,4.000000,14.862816,2.000000,...,0,0,0,0,0,0,0,0,0,0.000


In [403]:
df2.to_csv('../data/processed/data_traffic.csv', index=False)

In [404]:
df2.to_csv('data/processed/data_traffic.csv', index=False)

In [405]:
df2.to_csv('../src/data/data/processed/data_traffic.csv', index=False)